In [1]:
import librosa
import pandas as pd
import torch.optim as optim
import numpy as np
import csv
from data_load import TextTransform
from difflib import SequenceMatcher
import os
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from torch.functional import F
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
import torch
import torch.nn as nn
import librosa
from librosa.core import stft, magphase
from glob import glob
from torch import autograd
import csv
from data_load import CodeSwitchDataset
import zipfile

/home/dexter/Desktop/MSR challenge/hparam.py:11: YAMLLoadWarning: calling yaml.load_all() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  for doc in docs:


In [2]:
def pad(wav, trans, lang):
    if lang == "Gujarati":
        max_len = 0
    elif lang == "Telugu":
        max_len = 529862
    elif lang == 'Tamil':
        max_len = 0
    else:
        raise Exception("Check Language")

    while len(wav) < max_len:
        diff = max_len - len(wav)
        ext = wav[:diff]
        wav = np.append(wav, wav[:diff])
        ratio = int(len(trans)*diff/len(wav))
        trans +=trans[:ratio]
    return wav, trans

def preprocess(data):
    #print(data)
    inputs = []
    labels = []
    input_lengths = []
    label_lengths = []
    
    for (wav, sr, trans, lang) in data:
        #wav, trans  = pad(wav, trans, lang)
        out = stft(wav, win_length=int(sr*0.02), hop_length=int(sr*0.01))
        out = np.transpose(out, axes=(1, 0))

        text_transform = TextTransform()
        trans = torch.Tensor(text_transform.text_to_int(trans.lower()))

        out = magphase(out)[0]
        out = torch.from_numpy(np.array([np.log(1 + x) for x in out]))
        inputs.append(out)
        labels.append(trans)
        input_lengths.append(out.shape[0])
        label_lengths.append(len(trans))
    inputs = nn.utils.rnn.pad_sequence(inputs, batch_first=True)
    labels = nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=0)
    #spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
    return inputs, labels, input_lengths, label_lengths

In [3]:
def GreedyDecoder(output, labels, label_lengths, blank_label=3, collapse_repeated=True):
    arg_maxes = torch.argmax(output, dim=2)
    decodes = []
    text_transform = TextTransform()
    targets = []
    for i, args in enumerate(arg_maxes):
        decode = []
        targets.append(text_transform.int_to_text(labels[i][:label_lengths[i]].tolist()))
        for j, index in enumerate(args):
            if index != blank_label:
                if collapse_repeated and j != 0 and index == args[j -1]:
                    continue
                decode.append(index.item())
        decodes.append(text_transform.int_to_text(decode))
    return decodes, targets

In [4]:
train_dataset = CodeSwitchDataset(lang = 'Telugu', mode = "train")
validation_split = .2
shuffle_dataset = True
random_seed= 42
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)


In [5]:
train_loader = DataLoader(train_dataset,
                          batch_size=8,
                          drop_last=True,
                          num_workers = 6,
                          sampler = train_sampler,
                         collate_fn = lambda x: preprocess(x))
test_loader = DataLoader(train_dataset,
                          batch_size=8,
                          drop_last=True,
                          num_workers = 6,
                          sampler = valid_sampler,
                         collate_fn = lambda x: preprocess(x))

In [6]:
device = torch.device('cuda')

In [7]:
class Model(nn.Module):
    def __init__(self, input_dim, hidden_dim, batch_size, output_dim = 4, num_layers = 4):
        super(Model, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.output_dim = output_dim
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)
        #self.drop = nn.Dropout(0.25)
        self.linear = nn.Linear(self.hidden_dim, self.output_dim)
        
    def forward(self, x):
        #print(type(self.lstm))
        #print(x.shape)
        lstm_out, hidden = self.lstm(x)
        #lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        out = self.linear(lstm_out)
        return out, hidden

In [8]:
torch.cuda.empty_cache()

In [9]:
def train(model, device, train_loader, criterion, optimizer, scheduler, epoch, iter_meter, writer):
    model.train()
    data_len = len(train_loader.dataset)
    total_loss=0
    train_loss=0
    avg_acc=0
    acc=[]
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for batch_idx, (_data) in pbar:
        #bi, wav, label = batch_idx, wav, label
        wav, labels, input_lengths, label_lengths = _data
        wav = wav.to(device)
        wav = wav.float()
        labels = labels.to(device)
        optimizer.zero_grad()
        #print('Num Model Parameters', sum([param.nelement() for param in model.parameters()]))

        output, _ = model(wav)
        output = F.log_softmax(output, dim=1)
        output = output.transpose(0,1)
        #print(output.shape)
        loss = criterion(output, labels, input_lengths, label_lengths)
        #print(loss)
        total_loss+=loss
        loss.backward()
        
        optimizer.step()
        scheduler.step()
        iter_meter.step()
        decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)
        decoded_preds, decoded_targets = list(map(str.strip, decoded_preds)), list(map(str.strip, decoded_targets))
        for j in range(len(decoded_preds)):
            s = SequenceMatcher(None, decoded_targets[j], decoded_preds[j])
            acc.append(s.ratio())

        avg_acc = sum(acc)/len(acc)
        writer.add_scalar("train_accuracy", avg_acc, epoch)
        writer.add_scalar('train_loss', train_loss, epoch)
        writer.add_scalar('CTCLoss', loss, epoch*len(train_loader)+1)
        writer.add_scalar('TLoss', total_loss, epoch*len(train_loader)+1)
        if batch_idx % 100 == 0 or batch_idx == data_len:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(wav), data_len,
                    100. * batch_idx / len(train_loader), loss.item()))
            print("Train Accuracy: {}, Train loss: {}".format(avg_acc, train_loss))
            
    print(decoded_preds[0])
    if (epoch+1)%2 == 0:
        model.eval().cpu()
        ckpt_model_filename = "ckpt_epoch_" + str(epoch+1) + "_batch_id_" + str(batch_idx+1) + ".pth"
        ckpt_model_path = os.path.join("checkpoints/", ckpt_model_filename)
        torch.save(model.state_dict(), ckpt_model_path)
        model.to(device).train()

In [10]:
def test(model, device, test_loader, criterion, epoch, writer):
    model.eval()
    training_loss, train_acc = 0, 0
    eer, total_eer = 0, 0
    test_loss=0
    acc = []
    with torch.no_grad():
        for batch_idx, _data in enumerate(test_loader):
            inputs, labels, input_lengths, label_lengths = _data 
            inputs, labels = inputs.to(device), labels.to(device)

            output, _ = model(inputs)  # (batch, time, n_class)
            output = F.log_softmax(output, dim=1)
            output = output.transpose(0, 1) # (time, batch, n_class)
            loss = criterion(output, labels, input_lengths, label_lengths)
            test_loss += loss.item() / len(test_loader)
            
            decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)
            decoded_preds, decoded_targets = list(map(str.strip, decoded_preds)), list(map(str.strip, decoded_targets))
            for j in range(len(decoded_preds)):
                s = SequenceMatcher(None, decoded_targets[j], decoded_preds[j])
                acc.append(s.ratio())

            avg_acc = sum(acc)/len(acc)
            writer.add_scalar("test_accuracy", avg_acc, epoch)
            writer.add_scalar('test_loss', test_loss, epoch)
            print("Test Accuracy: {}, Test loss: {}".format(avg_acc, test_loss))
        
    print(decoded_targets)
    print(decoded_preds)

In [11]:
class IterMeter(object):
    """keeps track of total iterations"""
    def __init__(self):
        self.val = 0

    def step(self):
        self.val += 1

    def get(self):
        return self.val

In [12]:
model = Model(input_dim=1025,
              hidden_dim=512,
              batch_size=8,
              output_dim=4,
              num_layers=4)
#model.half()
model = model.to(device)
criterion = nn.CTCLoss(blank = 3).to(device)
epochs = 40
optimizer = optim.Adam(model.parameters(), 1e-3)

scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-3, 
                                            steps_per_epoch=int(len(train_loader)),
                                            epochs=40,
                                            anneal_strategy='linear')

In [ ]:
writer = SummaryWriter('train_logs/')
iter_meter = IterMeter()
#checkpoint = torch.load("checkpoints/final_epoch_2_batch_id_18.model")
#model.load_state_dict(checkpoint['model_state_dict'])
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#epoch = checkpoint['epoch']
model.to(device)
model.train()
for epoch in range(1, epochs+1):
    train(model, device, train_loader, criterion, optimizer, scheduler, epoch, iter_meter, writer)
    test(model, device, test_loader, criterion, epoch, writer)
    
model.eval().cpu()
save_model_filename = "final_epoch_" + str(epoch + 1) + "_batch_id_" + str(batch_idx + 1) + ".model"
save_model_path = os.path.join("checkpoints", save_model_filename)
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, save_model_path)

print("\nDone, trained model saved at", save_model_path)

  0%|          | 0/822 [00:00<?, ?it/s]

tensor([[ 3.1045e-07,  3.0678e-07, -2.4049e-07,  ..., -9.3601e-08,
         -4.5853e-07, -8.9079e-08],
        [-1.5973e-07,  1.3636e-07, -2.1445e-07,  ...,  2.6362e-07,
         -1.5314e-07, -2.4792e-08],
        [ 4.4377e-06, -3.9063e-07,  4.2341e-06,  ..., -2.0325e-06,
          2.9496e-06, -1.2188e-06],
        ...,
        [-2.4556e-07,  3.6336e-07, -7.9342e-07,  ...,  1.8261e-07,
         -8.3523e-07, -1.6329e-07],
        [-7.6132e-07,  4.0157e-07, -7.5902e-07,  ...,  6.4267e-07,
         -8.5748e-07,  1.2092e-07],
        [ 1.0603e-07,  1.1664e-08,  1.7802e-08,  ..., -6.8998e-08,
          3.1344e-08,  1.1882e-08]], device='cuda:0')


  0%|          | 1/822 [00:03<42:33,  3.11s/it]

Train Epoch: 1 [0/8225 (0%)]	Loss: 131.252884
Train Accuracy: 0.12132355868565889, Test loss: 0
tensor([[-2.7510e-07,  1.3603e-07, -4.1241e-07,  ...,  6.5741e-08,
         -4.0552e-07, -5.7732e-08],
        [-4.4967e-08, -1.0831e-07, -1.6284e-07,  ...,  3.2599e-07,
          4.2272e-08,  1.7776e-07],
        [ 1.8649e-06,  3.3944e-07,  3.7340e-06,  ..., -2.1598e-06,
          2.4650e-06, -7.4309e-07],
        ...,
        [-3.4374e-07,  4.6675e-07, -4.4612e-07,  ...,  1.4014e-07,
         -9.5086e-07, -5.7467e-07],
        [-1.9039e-07, -1.6689e-07, -7.8080e-07,  ...,  6.9098e-07,
         -6.1974e-07,  3.3473e-07],
        [-2.9921e-07, -1.1882e-07, -2.6946e-08,  ...,  2.6714e-07,
          2.6070e-07, -1.4795e-08]], device='cuda:0')


  0%|          | 2/822 [00:04<34:32,  2.53s/it]

...

tensor([[ 4.1066e-05,  2.6941e-05,  1.4064e-05,  ...,  3.8526e-05,
         -4.2331e-05, -3.6239e-05],
        [ 6.3188e-05,  1.1070e-04,  7.3192e-05,  ..., -1.5228e-05,
         -8.0149e-05, -5.7258e-05],
        [ 1.1880e-04,  1.2717e-04,  6.6317e-05,  ..., -6.8983e-05,
         -8.2247e-05, -4.3094e-05],
        ...,
        [ 4.0992e-04,  7.4677e-04,  6.1066e-04,  ..., -2.3923e-04,
         -3.4945e-04, -3.5783e-04],
        [-4.9232e-04, -1.1616e-03, -9.9345e-04,  ...,  8.2450e-04,
          6.3764e-04,  4.7632e-04],
        [-5.9732e-04, -1.3160e-03, -1.1202e-03,  ...,  8.7613e-04,
          7.8082e-04,  6.1954e-04]], device='cuda:0')


 13%|█▎        | 107/822 [01:11<08:31,  1.40it/s]

tensor([[ 1.4275e-03,  1.8513e-03,  7.6391e-04,  ...,  1.3893e-03,
         -1.9893e-03, -1.5804e-03],
        [-3.3138e-04, -3.8348e-04, -1.1924e-04,  ..., -4.9169e-04,
          5.9728e-04,  4.3426e-04],
        [-5.0811e-04, -6.1723e-04, -2.0530e-04,  ..., -1.3990e-03,
          1.6374e-03,  1.1736e-03],
        ...,
        [ 4.7078e-03,  6.7157e-03,  4.2025e-03,  ...,  8.0433e-07,
         -5.6894e-03, -4.8365e-03],
        [-1.0516e-02, -1.5263e-02, -1.0398e-02,  ...,  2.1292e-03,
          1.3992e-02,  1.1859e-02],
        [-1.2787e-02, -1.8404e-02, -1.2282e-02,  ...,  1.8296e-03,
          1.6882e-02,  1.4333e-02]], device='cuda:0')


 13%|█▎        | 108/822 [01:12<08:38,  1.38it/s]

tensor([[ 2.0489e-04,  3.2455e-04,  1.8461e-04,  ...,  7.3398e-05,
          1.4482e-05, -4.4129e-06],
        [ 9.4218e-05,  9.9144e-05,  4.6236e-05,  ..., -3.0824e-05,
         -7.2352e-05, -6.4789e-05],
        [ 2.8401e-04,  4.0210e-04,  2.0351e-04,  ..., -1.3015e-04,
         -2.4942e-04, -2.4094e-04],
        ...,
        [ 5.7069e-04,  9.8804e-04,  5.8972e-04,  ..., -2.7703e-04,
         -1.5465e-04, -1.7491e-04],
        [ 1.3690e-04, -1.4785e-04, -7.8573e-05,  ..., -1.3329e-05,
         -3.4632e-04, -2.5462e-04],
        [ 1.6008e-04, -2.5968e-04, -1.7253e-04,  ...,  1.7520e-05,
         -5.1740e-04, -3.9023e-04]], device='cuda:0')


In [27]:
data_len = len(train_loader.dataset)
pbar = tqdm(enumerate(train_loader), total=len(train_loader))
for batch_idx, (_data) in pbar:
    #bi, wav, label = batch_idx, wav, label
    wav, labels, input_lengths, label_lengths = _data
    print(wav.shape)




  0%|          | 0/822 [00:00<?, ?it/s]


  0%|          | 1/822 [00:02<36:46,  2.69s/it]

torch.Size([8, 1253, 1025])
torch.Size([8, 865, 1025])
torch.Size([8, 1265, 1025])
torch.Size([8, 757, 1025])
torch.Size([8, 1268, 1025])
torch.Size([8, 1719, 1025])


KeyboardInterrupt: 

In [14]:
checkpoint.keys()

NameError: name 'checkpoint' is not defined

In [28]:
labels

tensor([[0., 0., 2., 2., 2., 1., 1., 2., 2., 2., 2., 1., 1., 1., 1., 1., 1., 2.,
         2., 2., 2., 2., 0., 2., 2., 2., 2., 2., 2., 2., 2., 0., 0., 2., 2., 2.,
         2., 2., 2., 2., 2., 2., 2., 2., 1., 2., 2., 2., 2., 2., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 2., 2., 2., 2., 2., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2.,
         2., 2., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2., 2.,
         0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 1., 1., 2

In [29]:
input_lengths

[1020, 436, 1719, 618, 1023, 930, 810, 565]

In [30]:
inputs, labels, input_lengths, label_lengths = _data
inputs, labels = inputs.to(device), labels.to(device)

output, _ = model(inputs)  # (batch, time, n_class)
output = F.log_softmax(output, dim=1)
output = output.transpose(0, 1) # (time, batch, n_class)
loss = criterion(output, labels, input_lengths, label_lengths)

decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)

In [31]:
labels

tensor([[0., 0., 2., 2., 2., 1., 1., 2., 2., 2., 2., 1., 1., 1., 1., 1., 1., 2.,
         2., 2., 2., 2., 0., 2., 2., 2., 2., 2., 2., 2., 2., 0., 0., 2., 2., 2.,
         2., 2., 2., 2., 2., 2., 2., 2., 1., 2., 2., 2., 2., 2., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 2., 2., 2., 2., 2., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2.,
         2., 2., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2., 2.,
         0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 1., 1., 2

In [32]:
label_lengths

[51, 22, 86, 31, 51, 47, 41, 29]

In [33]:
input_lengths

[1020, 436, 1719, 618, 1023, 930, 810, 565]

In [34]:
decoded_preds[0], len(decoded_preds[0])

('sssssssssssssssssssssssssssssssssssstssssssssssssssssssssssssssssss', 67)

In [37]:
decoded_targets[0], len(decoded_targets[0])

('sstttttttttttttttttttstteeettteestttsttttttsssttttsttss', 55)

In [35]:
labels[0]

tensor([0., 0., 2., 2., 2., 1., 1., 2., 2., 2., 2., 1., 1., 1., 1., 1., 1., 2.,
        2., 2., 2., 2., 0., 2., 2., 2., 2., 2., 2., 2., 2., 0., 0., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 1., 2., 2., 2., 2., 2., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')

In [36]:
output.shape

torch.Size([1719, 8, 4])

In [37]:
inputs.shape

torch.Size([8, 1719, 1025])

In [38]:
for batch, dat in enumerate(train_dataset):
    inputs, labels, input_lengths, label_lengths = dat
    print(labels)

22050
22050
22050
22050
22050
22050
22050
22050
22050
22050
22050
22050
22050
22050
22050
22050
22050
22050
22050
22050
22050
22050
22050
22050
22050
22050
22050


KeyboardInterrupt: 

In [39]:
decoded_targets[0], len(decoded_targets[0])

('sstttssttttsssssstttttsttttttttsstttttttttttsttttts', 51)

In [40]:
for batch, dat in enumerate(train_loader):
    inputs, labels, input_lengths, label_lengths = dat
    inputs = inputs.to(device)
    output, _ = model(inputs)  # (batch, time, n_class)
    output = F.log_softmax(output, dim=1)
    output = output.transpose(0, 1) # (time, batch, n_class)
    loss = criterion(output, labels, input_lengths, label_lengths)

    decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)
    print(decoded_targets)
    print(decoded_preds)
    
    print(len(decoded_preds[0]))

['ssttttstttttttss', 'sstttsssssssttttttttssstttttttss', 'sstttttssttstttttttttttts', 'ssstttttssstttttttttttttttttttttsttttttsstttsss', 'ssttttttttssstttttttssstttttttss', 'ssttttttttttttttssstttttssttttttts', 'ssttttttttsttttttttttttsssttttttttssstttttttttts', 'ssstttttttttttttttttttssttttttss']
['ssssstssssss', 'sssssssssstssssssssssssssssssssssstsssss', 'ssttssstsstssstssttsssssststtssssssssssstssssss', 'ssssstsstststsssssssstsssssstssss', 'sssssstssssssssssssssssssssssssstssssss', 'sssssssstssstsssssssssssssssssssss', 'ssssssststssttsssssssssssssssstssssstssssssss', 'sssssssttssttssssssssssssssssstsssssssttssssssststststststtsssssssstssssssssstsssssts']
12
['sssttttttttttttttttttttstttts', 'sstttsttttssstttttttttttttsttsstttttsssstttstttss', 'sssssttttttttttttttttssttttss', 'sssttttssttstttttstttttss', 'ssttttttttttttttssttttts', 'sssssttsssssttttttttss', 'sssttttttsttttttttsstttttttss', 'sssttttttttttttttttsttttttttttsssssttttssstttttsssstsssssstttttttttsss']
['ssssssssssssssssss

['sssttttttssssstsssstttttttttssttttttttsssstttttttsttttttttss', 'sssssttssttsssssttssttttttsss', 'sssttttsssssttttttstttttttts', 'sstttttsstttttttttttss', 'ssttttsstttttttttttttss', 'sstttttttsssttttssssssstttttttttttsssstttttttssstttttttsssss', 'sssstttttttttssttssttttttts', 'sstttttttttssttttttsssttttttttttttss']
['ssssssstsssssssssssssststssssssssssssssssstsstssssssssssstsstssssssssssssssstsssssssssssssssssstsss', 'sstsssssssssssssssssssssssssssssssssssssssssssssssssssstsstsssssssstssstsssststsstsssssssssssssssttsssss', 'sssssssstsssssssssssstsssssssssstsssssssssssssstsssssstssttttssttssssststtsstssssstttstssstsssssssssssttststsssssssssssssssts', 'sssssssttsssssssssssssssssttttsssssttsssssststsssstsssssssstsssssssssssss', 'sssssssssssssstsststsssssssssssssssssssss', 'stsssssssssststsssssssss', 'stsssssttsstssssssss', 'stsstssttttsssssssss']
99
['ssttttttssstttssstttsssssssssstttttttttttts', 'sssssstttttttttstttttttssssstttttttttssssttttttttttss', 'ssstttttsstttttttssssttttttttttttt

KeyboardInterrupt: 

In [48]:
labels

tensor([[1., 1., 1., 2., 2., 3., 3., 3., 3., 3., 3., 2., 3., 2., 2., 3., 3., 2.,
         2., 3., 3., 3., 3., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 2., 2.,
         2., 3., 3., 3., 3., 3., 3., 3., 1., 1., 3., 3., 3., 3., 3., 3., 3., 3.,
         3., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
         3., 2., 2., 2., 2., 3., 3., 3., 3., 3., 3., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 3., 3., 3., 3., 3., 2., 3., 3., 3., 3., 3., 2., 2., 3., 3., 3.,
         2., 3., 3., 3., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1.,